In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/sound_depression.db'
mask_file = '/home/eli/Documents/sound_depression/ds000171-download/all_tasks/wholebrain.nii.gz'

OVERWRITE = True

In [4]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)

if not existed or OVERWRITE:
    data_files = glob.glob('/home/eli/Documents/sound_depression/fmriprep/sub-*/func/*_preproc.nii.gz')
    regex = re.compile("sub-([A-Za-z]+)([0-9]{1,2})_task-([A-Za-z]+)(_run-[0-9]{1,2})?_\S+preproc.nii.gz")
    for data_file in data_files:
        m = regex.search(data_file)
        condition, subject, task, run = m.groups()
        if run is None:
            run = ''
        tsv_file = '/sub-%s%s_task-%s%s_events.tsv' % (condition, subject, task, run)
        tsv_path = '/home/eli/Documents/sound_depression/ds000171-download/sub-%s%s/func/' % (condition, subject)
        tsv_file = os.path.dirname(tsv_path) + tsv_file
        with open(tsv_file, 'r') as tsv:
            tsv_reader = csv.DictReader(tsv, dialect='excel-tab')
            onset, duration, stimfile = [row for row in tsv_reader][0].values()
            onset = round(float(onset))
            duration = round(float(duration))
        
        logging.info("Recording blocks in %s", data_file)
       
        task_block = niidb.FMriActivationBlock(zscore=True)
        task_block.task = stimfile
        task_block.filename = data_file
        task_block.subject = subject
        task_block.start_time = onset
        task_block.end_time = onset + duration
        task_block.individual_differences = {'condition': condition}
        db.upsert(task_block)
        logging.info('Stimulus from %d to %d', onset, onset + duration)

12/28/2018 11:03:01 Recording blocks in /home/eli/Documents/sound_depression/fmriprep/sub-mdd07/func/sub-mdd07_task-music_run-2_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
12/28/2018 11:03:01 Stimulus from 0 to 33
12/28/2018 11:03:01 Recording blocks in /home/eli/Documents/sound_depression/fmriprep/sub-mdd07/func/sub-mdd07_task-nonmusic_run-4_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
12/28/2018 11:03:01 Stimulus from 0 to 33
12/28/2018 11:03:01 Recording blocks in /home/eli/Documents/sound_depression/fmriprep/sub-mdd07/func/sub-mdd07_task-music_run-1_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
12/28/2018 11:03:01 Stimulus from 0 to 33
12/28/2018 11:03:01 Recording blocks in /home/eli/Documents/sound_depression/fmriprep/sub-mdd07/func/sub-mdd07_task-music_run-3_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
12/28/2018 11:03:01 Stimulus from 0 to 33
12/28/2018 11:03:01 Recording blocks in /home/eli/Documents/sound_depression/fmriprep/sub-mdd07/func/sub-mdd07_task-nonmusic_run-5_bo

In [5]:
logging.info('Finished building NiiDb out of Lepping 2017 dataset')

12/28/2018 11:03:15 Finished building NiiDb out of Lepping 2017 dataset
